# Assignment 2 - Income Prediction

Let's do the BINARY CLASSIFICATION !!!

https://ntumlta.github.io/2017fall-ml-hw2/

Dataset : ADULT Dataset
Raw Data
* [train.csv](https://ntumlta.github.io/2017fall-ml-hw2/raw_data/train.csv)
* [test.csv](https://ntumlta.github.io/2017fall-ml-hw2/raw_data/test.csv)
* [Ref](https://archive.ics.uci.edu/ml/datasets/Adult)

Feature
* [X_train](https://ntumlta.github.io/2017fall-ml-hw2/feature/X_train)
    * 106 dims
    * one-hot encoding
* [Y_train](https://ntumlta.github.io/2017fall-ml-hw2/feature/Y_train)
    * label = 0 表示小於等於50K, label = 1 表示大於50K
* [X_test](https://ntumlta.github.io/2017fall-ml-hw2/feature/X_test)

Ans
* [Answer](https://ntumlta.github.io/2017fall-ml-hw2/correct_answer.csv)


# Logistic Regression
### import library

In [3]:
import os, sys
import numpy as np
from random import shuffle
import argparse
from math import log, floor
import pandas as pd

### IO File

In [4]:
def load_data(train_data_path, train_label_path, test_data_path):
    X_train = pd.read_csv(train_data_path, sep=',', header=0)
    X_train = np.array(X_train.values)
    Y_train = pd.read_csv(train_label_path, sep=',', header=0)
    Y_train = np.array(Y_train.values)
    X_test = pd.read_csv(test_data_path, sep=',', header=0)
    X_test = np.array(X_test.values)

    return (X_train, Y_train, X_test)

In [5]:
X_all_origin, Y_all, X_test_origin = load_data('data/X_train', 'data/Y_train', 'data/X_test')

### define shuffle

In [6]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
    return (X[randomize], Y[randomize])

### define normalize

In [15]:
def normalize(X_all, X_test):
    # Feature normalization with train and test X
    X_train_test = np.concatenate((X_all, X_test))
    mu = (sum(X_train_test) / X_train_test.shape[0])
    sigma = np.std(X_train_test, axis=0)
    mu = np.tile(mu, (X_train_test.shape[0], 1))
    sigma = np.tile(sigma, (X_train_test.shape[0], 1))
    X_train_test_normed = (X_train_test - mu) / sigma

    # Split to train, test again
    X_all = X_train_test_normed[0:X_all.shape[0]]
    X_test = X_train_test_normed[X_all.shape[0]:]
    return X_all, X_test

In [16]:
X_all, X_test = normalize(X_all_origin, X_test_origin)

### define split valid

In [20]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid

### define sigmoid

In [21]:
def sigmoid(z):
    res = 1 / (1.0 + np.exp(-z))
    return np.clip(res, 1e-8, 1-(1e-8))

### get valid score

In [22]:
def valid(w, b, X_valid, Y_valid):
    valid_data_size = len(X_valid)

    z = (np.dot(X_valid, np.transpose(w)) + b)
    y = sigmoid(z)
    y_ = np.around(y)
    result = (np.squeeze(Y_valid) == y_)
    print('Validation acc = %f' % (float(result.sum()) / valid_data_size))
    return

### train model

In [23]:
def train(X_all, Y_all, save_dir):
    # Split a 10%-validation set from the training set
    valid_set_percentage = 0.1
    X_train, Y_train, X_valid, Y_valid = split_valid_set(X_all, Y_all, valid_set_percentage)

    # Initiallize parameter, hyperparameter
    w = np.zeros((106,))
    b = np.zeros((1,))
    l_rate = 0.1
    batch_size = 32
    train_data_size = len(X_train)
    step_num = int(floor(train_data_size / batch_size))
    epoch_num = 1000
    save_param_iter = 50

    # Start training
    total_loss = 0.0
    for epoch in range(1, epoch_num):
        # Do validation and parameter saving
        if (epoch) % save_param_iter == 0:
            print('=====Saving Param at epoch %d=====' % epoch)
            if not os.path.exists(save_dir):
                os.mkdir(save_dir)
            np.savetxt(os.path.join(save_dir, 'w'), w)
            np.savetxt(os.path.join(save_dir, 'b'), [b,])
            print('epoch avg loss = %f' % (total_loss / (float(save_param_iter) * train_data_size)))
            total_loss = 0.0
            valid(w, b, X_valid, Y_valid)

        # Random shuffle
        X_train, Y_train = _shuffle(X_train, Y_train)

        # Train with batch
        for idx in range(step_num):
            X = X_train[idx*batch_size:(idx+1)*batch_size]
            Y = Y_train[idx*batch_size:(idx+1)*batch_size]

            z = np.dot(X, np.transpose(w)) + b
            y = sigmoid(z)

            cross_entropy = -1 * (np.dot(np.squeeze(Y), np.log(y)) + np.dot((1 - np.squeeze(Y)), np.log(1 - y)))
            total_loss += cross_entropy

            w_grad = np.mean(-1 * X * (np.squeeze(Y) - y).reshape((batch_size,1)), axis=0)
            b_grad = np.mean(-1 * (np.squeeze(Y) - y))

            # SGD updating parameters
            w = w - l_rate * w_grad
            b = b - l_rate * b_grad

    return

In [24]:
train(X_all, Y_all, 'data/save')

=====Saving Param at epoch 50=====
epoch avg loss = 0.301514
Validation acc = 0.844532
=====Saving Param at epoch 100=====
epoch avg loss = 0.302840
Validation acc = 0.844156
=====Saving Param at epoch 150=====
epoch avg loss = 0.303154
Validation acc = 0.843713
=====Saving Param at epoch 200=====
epoch avg loss = 0.303048
Validation acc = 0.844293
=====Saving Param at epoch 250=====
epoch avg loss = 0.302794
Validation acc = 0.843269
=====Saving Param at epoch 300=====
epoch avg loss = 0.302814
Validation acc = 0.844054
=====Saving Param at epoch 350=====
epoch avg loss = 0.302646
Validation acc = 0.843133
=====Saving Param at epoch 400=====
epoch avg loss = 0.302869
Validation acc = 0.842894
=====Saving Param at epoch 450=====
epoch avg loss = 0.302724
Validation acc = 0.843849
=====Saving Param at epoch 500=====
epoch avg loss = 0.302744
Validation acc = 0.844361
=====Saving Param at epoch 550=====
epoch avg loss = 0.302884
Validation acc = 0.844498
=====Saving Param at epoch 600===

### infer & output ans.csv

In [25]:
def infer(X_test, save_dir, output_dir):
    test_data_size = len(X_test)

    # Load parameters
    print('=====Loading Param from %s=====' % save_dir)
    w = np.loadtxt(os.path.join(save_dir, 'w'))
    b = np.loadtxt(os.path.join(save_dir, 'b'))

    # predict
    z = (np.dot(X_test, np.transpose(w)) + b)
    y = sigmoid(z)
    y_ = np.around(y)

    print('=====Write output to %s =====' % output_dir)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    output_path = os.path.join(output_dir, 'log_prediction.csv')
    with open(output_path, 'w') as f:
        f.write('id,label\n')
        for i, v in  enumerate(y_):
            f.write('%d,%d\n' %(i+1, v))

    return

In [26]:
infer(X_test, 'data/save', 'data/output')

=====Loading Param from data/save=====
=====Write output to data/output =====
